<h1><center>Tutorial - Gassmann Fluid Substitution</center></h1>

Fluid substitution is an important part of the seismic rock physics analysis (e.g., AVO, 4D analysis), which provides
a tool for fluid identification and quantification in reservoir. This is commonly performed using Gassmann’s equation (Gassmann, 1951).

In [1]:
#Import Libraries
from pyavo.avotools import gassmann

__1. create model parameters__

In [2]:
# vp = P-wave velocity from log (ft/s)
# vs = S-wave velocity from log (ft/s)
# rho = Bulk density form log (g/cc)
# rho_o = Oil gravity (deg API)
# rho_g = Gas gravity (API)
# vsh = Shale volume from log
# phi = Porosity
# swi = Initial water saturation from log
# swt =  Target water saturation
# S = Salinity (ppm)
# T = Temperature (deg)
# P = Pressure (psi)
# init_fluid = Fluid type of initial hydrocarbon (gas or oil)
# final_fluid = Fluid type of desired output where (gas or oil)
# GOR = Gas-Oil ratio

__2. Initiate the class to model Gassmann fluid substitution for brine sands, oil sands, and gas sands.__

Desired fluid is defined by the target water saturation (swt) and the type of hydrocarbon. Here, we assume swt to be 80%

In [3]:
model = gassmann.GassmannSub(vp=11000, vs=6500, rho=2.2, rho_g=0.9, rho_o=42, vsh=0.2, 
                    phi=0.2, swi=0.4, swt=0.8, S=3800, T=150, P=3200, init_fluid='oil', 
                    final_fluid='gas', GOR=160.0)

__3. Estimate the bulk modulus and density of matrix__

In [4]:
k_mat, rho_mat = gassmann.k_rho_matrix(v_cly=0.14, k_cly=20.9, k_qtz=36.6, rho_cly=2.58, rho_qtz=2.65)
print(f'Bulk Modulus of matrix: {k_mat}Gpa')
print(f'Density of matrix: {rho_mat}g/cc')

Bulk Modulus of matrix: 33.76Gpa
Density of matrix: 2.64g/cc


__4 . Estimate bulk modulus and density of brine__

In [5]:
k_brine, rho_brine = model.k_rho_brine()
print(f'Bulk Modulus of brine: {k_brine}Gpa')
print(f'Density of brine: {rho_brine}g/cc')

Bulk Modulus of brine: 2.663203705824249e-05Gpa
Density of brine: 0.933g/cc


__5. Estimate the bulk modulus and density of initial hydrocarbon__

In [6]:
k_hyc_init, rho_hyc_init = model.init_hyc()
print(f'Bulk Modulus of initial hydrocarbon: {k_hyc_init}Gpa')
print(f'Density of initial hydrocarbon: {rho_hyc_init}g/cc')

Bulk Modulus of initial hydrocarbon: 0.2199Gpa
Density of initial hydrocarbon: 0.553g/cc


__6. Estimate bulk modulus and density of initial fluid__

In [7]:
k_fl, rho_fl = model.k_rho_fluid()
print(f'Bulk Modulus of initial fluid: {k_fl}Gpa')
print(f'Density of initial fluid: {rho_fl}g/cc')

Bulk Modulus of initial fluid: 0.37Gpa
Density of initial fluid: 0.71g/cc


__7. Estimate initial saturated bulk modulus and shear modulus__

In [8]:
k_sat_init, mu_sat_init = model.insitu_moduli(rho_fluid=rho_fl, rho_matrix=rho_mat, d_phi=True)
print(f'Initial saturated bulk modulus: {k_sat_init}Gpa')
print(f'Initial saturated shear modulus: {mu_sat_init}Gpa')

Initial saturated bulk modulus: 13.2344Gpa
Initial saturated shear modulus: 8.6467Gpa


__8. Estimate frame bulk modulus__

In [9]:
k_frame = model.k_frame(k_mat=k_mat, k_fld=k_fl, k_sat=k_sat_init)
print(f'Frame bulk modulus: {k_frame}Gpa')

Frame bulk modulus: 12.5189Gpa


__9. Estimate bulk modulus and density of the desired hydrocarbon__

In [10]:
k_hyc, rho_hyc = model.final_hc()
print(f'Bulk modulus of desired hydrocarbon: {k_hyc}Gpa')
print(f'Density of desired hydrocarbon: {rho_hyc}g/cc')

Bulk modulus of desired hydrocarbon: 0.05307560388919129Gpa
Density of desired hydrocarbon: 0.18189858127346814g/cc


__10. Estimate the bulk modulus and density of saturated rock after fluid substitution__

In [11]:
k_sat_new, rho_sat_new = model.k_rho_sat(k_mat=k_mat, rho_mat=rho_mat, k_frame=k_frame)
print(f'Bulk modulus of saturated rock after substitution: {k_sat_new}Gpa')
print(f'Density of saturated rock after substitution: {rho_sat_new}Gpa')

Bulk modulus of saturated rock after substitution: 12.519Gpa
Density of saturated rock after substitution: 2.269Gpa


__11. Esimate the seismic velocities__

In [12]:
vp_new, vs_new = gassmann.vel_sat(k_sat=k_sat_new, rho_sat=rho_sat_new, mu=mu_sat_init)
print(f'Density of Saturated rock after substitution: {rho_sat_new}g/cc')
print(f'P-wave velocity after fluid substitution {vp_new}m/s')
print(f'S-wave velocity after fluid substitution {vs_new}m/s')

Density of Saturated rock after substitution: 2.269g/cc
P-wave velocity after fluid substitution 10680.87m/s
S-wave velocity after fluid substitution 6404.61m/s
